## Problem description

In this kernel, we are going to use Densenet121 pretrained model with Pytorch.

## Libraries

In [1]:
import gc
import os
import sys
import time
import random
import logging
import datetime as dt

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F
import torchvision as vision

from torch.optim.lr_scheduler import CosineAnnealingLR

from pathlib import Path
from PIL import Image
from contextlib import contextmanager

from joblib import Parallel, delayed
from tqdm import tqdm
from fastprogress import master_bar, progress_bar

from sklearn.model_selection import KFold
from sklearn.metrics import fbeta_score

torch.multiprocessing.set_start_method("spawn")

## Utilities

In [2]:
@contextmanager
def timer(name="Main", logger=None):
    t0 = time.time()
    yield
    msg = f"[{name}] done in {time.time() - t0} s"
    if logger is not None:
        logger.info(msg)
    else:
        print(msg)
        

def get_logger(name="Main", tag="exp", log_dir="log/"):
    log_path = Path(log_dir)
    path = log_path / tag
    path.mkdir(exist_ok=True, parents=True)

    logger = logging.getLogger(name)
    logger.setLevel(logging.INFO)

    fh = logging.FileHandler(
        path / (dt.datetime.now().strftime("%Y-%m-%d-%H-%M-%S") + ".log"))
    sh = logging.StreamHandler(sys.stdout)
    formatter = logging.Formatter(
        "%(asctime)s %(name)s %(levelname)s %(message)s")

    fh.setFormatter(formatter)
    sh.setFormatter(formatter)
    logger.addHandler(fh)
    logger.addHandler(sh)
    return logger


def seed_torch(seed=1029):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

In [3]:
logger = get_logger(name="Main", tag="Pytorch-VGG16")

## Data Loading

In [4]:
!ls ../input/imet-2019-fgvc6/

labels.csv  sample_submission.csv  test  train	train.csv


In [5]:
labels = pd.read_csv("../input/imet-2019-fgvc6/labels.csv")
train = pd.read_csv("../input/imet-2019-fgvc6/train.csv")
sample = pd.read_csv("../input/imet-2019-fgvc6/sample_submission.csv")
train.head()

,id,attribute_ids
0,1000483014d91860,147 616 813
1,1000fe2e667721fe,51 616 734 813
2,1001614cb89646ee,776
3,10041eb49b297c08,51 671 698 813 1092
4,100501c227f8beea,13 404 492 903 1093


In [6]:
!cp ../input/pytorch-pretrained-image-models/* ./
!ls

__notebook_source__.ipynb  densenet201.pth  resnet34.pth
densenet121.pth		   log		    resnet50.pth


## DataLoader

In [7]:
# This loader is to extract 1024d features from the images.
class ImageDataLoader(data.DataLoader):
    def __init__(self, root_dir: Path, 
                 df: pd.DataFrame, 
                 mode="train", 
                 transforms=None):
        self._root = root_dir
        self.transform = transforms[mode]
        self._img_id = (df["id"] + ".png").values
        
    def __len__(self):
        return len(self._img_id)
    
    def __getitem__(self, idx):
        img_id = self._img_id[idx]
        file_name = self._root / img_id
        img = Image.open(file_name)
        
        if self.transform:
            img = self.transform(img)
            
        return [img]
    
    
data_transforms = {
    'train': vision.transforms.Compose([
        vision.transforms.RandomResizedCrop(224),
        vision.transforms.RandomHorizontalFlip(),
        vision.transforms.ToTensor(),
        vision.transforms.Normalize(
            [0.485, 0.456, 0.406], 
            [0.229, 0.224, 0.225])
    ]),
    'val': vision.transforms.Compose([
        vision.transforms.Resize(256),
        vision.transforms.CenterCrop(224),
        vision.transforms.ToTensor(),
        vision.transforms.Normalize(
            [0.485, 0.456, 0.406], 
            [0.229, 0.224, 0.225])
    ]),
}

data_transforms["test"] = data_transforms["val"]

In [8]:
# This loader is to be used for serving image tensors for the MLP.
class IMetDataset(data.Dataset):
    def __init__(self, tensor, device="cuda:0", labels=None):
        self.tensor = tensor
        self.labels = labels
        self.device= device
        
    def __len__(self):
        return self.tensor.size(0)
    
    def __getitem__(self, idx):
        tensor = self.tensor[idx, :]
        if self.labels is not None:
            label = self.labels[idx]
            label_tensor = torch.zeros((1, 1103))
            for i in label:
                label_tensor[0, int(i)] = 1
            label_tensor = label_tensor.to(self.device)
            return [tensor, label_tensor]
        else:
            return [tensor]

## Model

Since passing images through the Densenet121 takes a lot of time, I don't want to take that time per epoch.

I will calculate the output of the Densenet121 beforehand so that converting raw images to processed feature vectors happens once.

In [9]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        
    def forward(self, x):
        return x


class Densenet121(nn.Module):
    def __init__(self, pretrained: Path):
        super(Densenet121, self).__init__()
        self.densenet121 = vision.models.densenet121()
        self.densenet121.load_state_dict(torch.load(pretrained))
        self.densenet121.classifier = Classifier()
        
        dense = nn.Sequential(*list(self.densenet121.children())[:-1])
        for param in dense.parameters():
            param.requires_grad = False
        
    def forward(self, x):
        return self.densenet121(x)
    
    
class MultiLayerPerceptron(nn.Module):
    def __init__(self):
        super(MultiLayerPerceptron, self).__init__()
        self.linear1 = nn.Linear(1024, 1024)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(1024, 1103)
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.relu(self.linear1(x))
        x = self.dropout(x)
        return self.sigmoid(self.linear2(x))

## Data Preprocessing with Densenet121

In [10]:
train_dataset = ImageDataLoader(
    root_dir=Path("../input/imet-2019-fgvc6/train/"),
    df=train,
    mode="train",
    transforms=data_transforms)
train_loader = data.DataLoader(dataset=train_dataset,
                               shuffle=False,
                               batch_size=128)
test_dataset = ImageDataLoader(
    root_dir=Path("../input/imet-2019-fgvc6/test/"),
    df=sample,
    mode="test",
    transforms=data_transforms)
test_loader = data.DataLoader(dataset=test_dataset,
                              shuffle=False,
                              batch_size=128)

In [11]:
def get_feature_vector(df, loader, device):
    matrix = torch.zeros((df.shape[0], 1024)).to(device)
    model = Densenet121("densenet121.pth")
    model.to(device)
    batch = loader.batch_size
    for i, (i_batch,) in tqdm(enumerate(loader)):
        i_batch = i_batch.to(device)
        pred = model(i_batch).detach()
        matrix[i * batch:(i + 1) * batch] = pred
    return matrix

In [12]:
train_tensor = get_feature_vector(train, train_loader, "cuda:0")
test_tensor = get_feature_vector(sample, test_loader, "cuda:0")

854it [29:09,  1.71s/it]
59it [01:55,  1.48s/it]


In [13]:
del train_dataset, train_loader
del test_dataset, test_loader
gc.collect()

868

## Train Utilities

In [14]:
class Trainer:
    def __init__(self, 
                 model, 
                 logger,
                 n_splits=5,
                 seed=42,
                 device="cuda:0",
                 train_batch=32,
                 valid_batch=128,
                 kwargs={}):
        self.model = model
        self.logger = logger
        self.device = device
        self.n_splits = n_splits
        self.seed = seed
        self.train_batch = train_batch
        self.valid_batch = valid_batch
        self.kwargs = kwargs
        
        self.best_score = None
        self.tag = dt.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
        self.loss_fn = nn.BCELoss(reduction="mean").to(self.device)
        
        path = Path(f"bin/{self.tag}")
        path.mkdir(exist_ok=True, parents=True)
        self.path = path
        
    def fit(self, X, y, n_epochs=10):
        train_preds = np.zeros((len(X), 1103))
        fold = KFold(n_splits=self.n_splits, random_state=self.seed)
        for i, (trn_idx, val_idx) in enumerate(fold.split(X)):
            self.fold_num = i
            self.logger.info(f"Fold {i + 1}")
            X_train, X_val = X[trn_idx, :], X[val_idx, :]
            y_train, y_val = y[trn_idx], y[val_idx]
            
            valid_preds = self._fit(X_train, y_train, X_val, y_val, n_epochs)
            train_preds[val_idx] = valid_preds
        return train_preds
    
    def _fit(self, X_train, y_train, X_val, y_val, n_epochs):
        seed_torch(self.seed)
        train_dataset = IMetDataset(X_train, labels=y_train, device=self.device)
        train_loader = data.DataLoader(train_dataset, 
                                       batch_size=self.train_batch,
                                       shuffle=True)

        valid_dataset = IMetDataset(X_val, labels=y_val, device=self.device)
        valid_loader = data.DataLoader(valid_dataset,
                                       batch_size=self.valid_batch,
                                       shuffle=False)
        
        model = self.model(**self.kwargs)
        model.to(self.device)
        
        optimizer = optim.Adam(params=model.parameters(), 
                                lr=0.0001)
        scheduler = CosineAnnealingLR(optimizer, T_max=n_epochs)
        best_score = np.inf
        mb = master_bar(range(n_epochs))
        for epoch in mb:
            model.train()
            avg_loss = 0.0
            for i_batch, y_batch in progress_bar(train_loader, parent=mb):
                y_pred = model(i_batch)
                loss = self.loss_fn(y_pred, y_batch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                avg_loss += loss.item() / len(train_loader)
            valid_preds, avg_val_loss = self._val(valid_loader, model)
            scheduler.step()

            self.logger.info("=========================================")
            self.logger.info(f"Epoch {epoch + 1} / {n_epochs}")
            self.logger.info("=========================================")
            self.logger.info(f"avg_loss: {avg_loss:.8f}")
            self.logger.info(f"avg_val_loss: {avg_val_loss:.8f}")
            
            if best_score > avg_val_loss:
                torch.save(model.state_dict(),
                           self.path / f"best{self.fold_num}.pth")
                self.logger.info(f"Save model at Epoch {epoch + 1}")
                best_score = avg_val_loss
        model.load_state_dict(torch.load(self.path / f"best{self.fold_num}.pth"))
        valid_preds, avg_val_loss = self._val(valid_loader, model)
        self.logger.info(f"Best Validation Loss: {avg_val_loss:.8f}")
        return valid_preds
    
    def _val(self, loader, model):
        model.eval()
        valid_preds = np.zeros((len(loader.dataset), 1103))
        avg_val_loss = 0.0
        for i, (i_batch, y_batch) in enumerate(loader):
            with torch.no_grad():
                y_pred = model(i_batch).detach()
                avg_val_loss += self.loss_fn(y_pred, y_batch).item() / len(loader)
                valid_preds[i * self.valid_batch:(i + 1) * self.valid_batch] = \
                    y_pred.cpu().numpy()
        return valid_preds, avg_val_loss
    
    def predict(self, X):
        dataset = IMetDataset(X, labels=None)
        loader = data.DataLoader(dataset, 
                                 batch_size=self.valid_batch, 
                                 shuffle=False)
        model = self.model(**self.kwargs)
        preds = np.zeros((X.size(0), 1103))
        for path in self.path.iterdir():
            with timer(f"Using {str(path)}", self.logger):
                model.load_state_dict(torch.load(path))
                model.to(self.device)
                model.eval()
                temp = np.zeros_like(preds)
                for i, (i_batch, ) in enumerate(loader):
                    with torch.no_grad():
                        y_pred = model(i_batch).detach()
                        temp[i * self.valid_batch:(i + 1) * self.valid_batch] = \
                            y_pred.cpu().numpy()
                preds += temp / self.n_splits
        return preds

## Training

In [15]:
trainer = Trainer(MultiLayerPerceptron, logger, train_batch=64, kwargs={})

In [16]:
y = train.attribute_ids.map(lambda x: x.split()).values
valid_preds = trainer.fit(train_tensor, y, n_epochs=40)

2022-12-06 20:30:04,822 Main INFO Fold 1


/opt/conda/lib/python3.6/site-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([64, 1, 1103])) that is different to the input size (torch.Size([64, 1103])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
/opt/conda/lib/python3.6/site-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([29, 1, 1103])) that is different to the input size (torch.Size([29, 1103])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
/opt/conda/lib/python3.6/site-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([128, 1, 1103])) that is different to the input size (torch.Size([128, 1103])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


2022-12-06 20:30:13,497 Main INFO =========================================
2022-12-06 20:30:13,500 Main INFO Epoch 1 / 40
2022-12-06 20:30:13,501 Main INFO =========================================
2022-12-06 20:30:13,504 Main INFO avg_loss: 0.02454177
2022-12-06 20:30:13,505 Main INFO avg_val_loss: 0.01283414
2022-12-06 20:30:13,516 Main INFO Save model at Epoch 1


/opt/conda/lib/python3.6/site-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([88, 1, 1103])) that is different to the input size (torch.Size([88, 1103])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


2022-12-06 20:30:22,239 Main INFO =========================================
2022-12-06 20:30:22,240 Main INFO Epoch 2 / 40
2022-12-06 20:30:22,244 Main INFO =========================================
2022-12-06 20:30:22,244 Main INFO avg_loss: 0.01283906
2022-12-06 20:30:22,247 Main INFO avg_val_loss: 0.01180768
2022-12-06 20:30:22,265 Main INFO Save model at Epoch 2
2022-12-06 20:30:31,211 Main INFO =========================================
2022-12-06 20:30:31,212 Main INFO Epoch 3 / 40
2022-12-06 20:30:31,214 Main INFO =========================================
2022-12-06 20:30:31,215 Main INFO avg_loss: 0.01196755
2022-12-06 20:30:31,216 Main INFO avg_val_loss: 0.01130774
2022-12-06 20:30:31,234 Main INFO Save model at Epoch 3
2022-12-06 20:30:39,530 Main INFO =========================================
2022-12-06 20:30:39,531 Main INFO Epoch 4 / 40
2022-12-06 20:30:39,532 Main INFO =========================================
2022-12-06 20:30:39,534 Main INFO avg_loss: 0.01146331
2022-12-

...

2022-12-06 20:41:48,433 Main INFO =========================================
2022-12-06 20:41:48,434 Main INFO Epoch 1 / 40
2022-12-06 20:41:48,436 Main INFO =========================================
2022-12-06 20:41:48,437 Main INFO avg_loss: 0.02453290
2022-12-06 20:41:48,438 Main INFO avg_val_loss: 0.01290982
2022-12-06 20:41:48,449 Main INFO Save model at Epoch 1


/opt/conda/lib/python3.6/site-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([87, 1, 1103])) that is different to the input size (torch.Size([87, 1103])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


2022-12-06 20:41:56,952 Main INFO =========================================
2022-12-06 20:41:56,952 Main INFO Epoch 2 / 40
2022-12-06 20:41:56,953 Main INFO =========================================
2022-12-06 20:41:56,955 Main INFO avg_loss: 0.01281260
2022-12-06 20:41:56,958 Main INFO avg_val_loss: 0.01185245
2022-12-06 20:41:56,974 Main INFO Save model at Epoch 2
2022-12-06 20:42:05,928 Main INFO =========================================
2022-12-06 20:42:05,929 Main INFO Epoch 3 / 40
2022-12-06 20:42:05,930 Main INFO =========================================
2022-12-06 20:42:05,934 Main INFO avg_loss: 0.01194475
2022-12-06 20:42:05,935 Main INFO avg_val_loss: 0.01135724
2022-12-06 20:42:05,951 Main INFO Save model at Epoch 3
2022-12-06 20:42:14,488 Main INFO =========================================
2022-12-06 20:42:14,489 Main INFO Epoch 4 / 40
2022-12-06 20:42:14,490 Main INFO =========================================
2022-12-06 20:42:14,491 Main INFO avg_loss: 0.01144783
2022-12-

2022-12-06 20:47:35,085 Main INFO =========================================
2022-12-06 20:47:35,086 Main INFO Epoch 1 / 40
2022-12-06 20:47:35,089 Main INFO =========================================
2022-12-06 20:47:35,090 Main INFO avg_loss: 0.02452138
2022-12-06 20:47:35,091 Main INFO avg_val_loss: 0.01297089
2022-12-06 20:47:35,103 Main INFO Save model at Epoch 1
2022-12-06 20:47:43,650 Main INFO =========================================
2022-12-06 20:47:43,651 Main INFO Epoch 2 / 40
2022-12-06 20:47:43,651 Main INFO =========================================
2022-12-06 20:47:43,653 Main INFO avg_loss: 0.01280638
2022-12-06 20:47:43,654 Main INFO avg_val_loss: 0.01190094
2022-12-06 20:47:43,669 Main INFO Save model at Epoch 2
2022-12-06 20:47:52,040 Main INFO =========================================
2022-12-06 20:47:52,041 Main INFO Epoch 3 / 40
2022-12-06 20:47:52,044 Main INFO =========================================
2022-12-06 20:47:52,045 Main INFO avg_loss: 0.01193970
2022-12-

2022-12-06 20:53:19,034 Main INFO =========================================
2022-12-06 20:53:19,035 Main INFO Epoch 1 / 40
2022-12-06 20:53:19,038 Main INFO =========================================
2022-12-06 20:53:19,038 Main INFO avg_loss: 0.02453817
2022-12-06 20:53:19,040 Main INFO avg_val_loss: 0.01293391
2022-12-06 20:53:19,051 Main INFO Save model at Epoch 1
2022-12-06 20:53:27,581 Main INFO =========================================
2022-12-06 20:53:27,582 Main INFO Epoch 2 / 40
2022-12-06 20:53:27,583 Main INFO =========================================
2022-12-06 20:53:27,584 Main INFO avg_loss: 0.01281094
2022-12-06 20:53:27,586 Main INFO avg_val_loss: 0.01189603
2022-12-06 20:53:27,599 Main INFO Save model at Epoch 2
2022-12-06 20:53:36,057 Main INFO =========================================
2022-12-06 20:53:36,058 Main INFO Epoch 3 / 40
2022-12-06 20:53:36,060 Main INFO =========================================
2022-12-06 20:53:36,062 Main INFO avg_loss: 0.01194750
2022-12-

## Post process - threshold search -

Since I used sigmoid for the activation, I've got the 1103 probability output for each data row.

I need to decide threshold for this.There are two ways to deal with this.

- Class-wise threshold search
  - Takes some time but it's natural.
- One threshold for all the class
  - Low cost way.

**UPDATE**
I will use the first -> second one.

In [17]:
def threshold_search(y_pred, y_true):
    score = []
    candidates = np.arange(0, 1.0, 0.01)
    for th in progress_bar(candidates):
        yp = (y_pred > th).astype(int)
        score.append(fbeta_score(y_pred=yp, y_true=y_true, beta=2, average="samples"))
    score = np.array(score)
    pm = score.argmax()
    best_th, best_score = candidates[pm], score[pm]
    return best_th, best_score

In [18]:
y_true = np.zeros((train.shape[0], 1103)).astype(int)
for i, row in enumerate(y):
    for idx in row:
        y_true[i, int(idx)] = 1

In [19]:
best_threshold, best_score = threshold_search(valid_preds, y_true)
best_score

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


0.4491267830707251

## Prediction for test data

In [20]:
test_preds = trainer.predict(test_tensor)

2022-12-06 21:12:23,005 Main INFO [Using bin/2022-12-06-20-30-04/best3.pth] done in 0.2578892707824707 s
2022-12-06 21:12:23,185 Main INFO [Using bin/2022-12-06-20-30-04/best0.pth] done in 0.178605318069458 s
2022-12-06 21:12:23,363 Main INFO [Using bin/2022-12-06-20-30-04/best2.pth] done in 0.17663335800170898 s
2022-12-06 21:12:23,540 Main INFO [Using bin/2022-12-06-20-30-04/best1.pth] done in 0.17629098892211914 s
2022-12-06 21:12:23,719 Main INFO [Using bin/2022-12-06-20-30-04/best4.pth] done in 0.17759060859680176 s


In [21]:
preds = (test_preds > best_threshold).astype(int)

In [22]:
prediction = []
for i in range(preds.shape[0]):
    pred1 = np.argwhere(preds[i] == 1.0).reshape(-1).tolist()
    pred_str = " ".join(list(map(str, pred1)))
    prediction.append(pred_str)
    
sample.attribute_ids = prediction
sample.to_csv("submission.csv", index=False)
sample.head()

,id,attribute_ids
0,10023b2cc4ed5f68,121 195 223 343 369 587 766 1059
1,100fbe75ed8fd887,93 231 369 1039
2,101b627524a04f19,79 180 420 482 497 498 728 784
3,10234480c41284c6,13 51 111 147 480 483 725 776 813 830 923 1046
4,1023b0e2636dcea8,147 189 194 322 477 489 584 612 671 780 813 95...
